In [6]:
import pandas as pd

In [7]:
df = pd.read_csv("results.csv")

In [ ]:
df.groupby(by="game_id").max()

,epoch,name,position,game_result,timestamp
game_id,,,,,
0,50,S,50,50,1745311709765
1,50,S,50,50,1745311709815
2,44,S,50,50,1745311709850
3,55,S,50,50,1745311709894
4,49,S,50,50,1745311709930
...,...,...,...,...,...
95,9,S,10,-1,1745311712821
96,6,S,9,-1,1745311712827
97,59,S,50,50,1745311712872
